In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
CCI Swing Backtester (NSE)
+ Optional EMA fast/slow & SMA(trend) filters with on/off toggles.

ENTRY (long)
------------
Base:       CCI(n) crosses UP through cci_buy_level (e.g., -100)
+ Optional: EMA_fast > EMA_slow                 [use_ema_cross_filter=True]
+ Optional: Close > SMA(sma_trend_len, default 200)  [use_sma_trend_filter=True]

EXIT (first hit)
----------------
• Hard Stop-Loss (percent of entry)
• Take-Profit    (percent of entry)
• Indicator exit: CCI crosses DOWN through cci_sell_level (e.g., 0)
• Optional: EMA_fast < EMA_slow                [exit_on_ema_cross_down=True]
• Optional: Close < SMA(sma_trend_len)         [exit_on_sma_trend_breach=True]
"""

import os, json, math, warnings, logging
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore", category=FutureWarning)

try:
    import yfinance as yf
except Exception:
    pass

# =========================
# LOGGING
# =========================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
log = logging.getLogger("cci_swing_ema_fastslow_smatrend_v3")

# =========================
# CONFIG
# =========================
@dataclass
class Config:
    # Data
    start_date: str = "2015-01-01"
    end_date: str   = "2025-01-01"
    static_symbols: Optional[List[str]] = None
    static_symbols_path: Optional[str] = None
    cache_dir: str = "cache"
    out_dir: str   = "outputs"
    plot: bool     = True

    # --- CCI (core) ---
    cci_len: int = 20
    cci_buy_level: float  = -100.0
    cci_sell_level: float = 0.0

    # --- NEW: EMAs & long SMA trend (all configurable) ---
    ema_fast: int = 10          # e.g., EMA10
    ema_slow: int = 20          # e.g., EMA20
    sma_trend_len: int = 200    # e.g., SMA200 trend

    # Toggles for using the above as filters/exits
    use_ema_cross_filter: bool = False          # require EMA_fast > EMA_slow at entry
    use_sma_trend_filter: bool = False          # require Close > SMA(sma_trend_len) at entry
    exit_on_ema_cross_down: bool = False       # exit if EMA_fast < EMA_slow
    exit_on_sma_trend_breach: bool = False     # exit if Close < SMA(sma_trend_len)

    # Fixed exits
    stop_loss_pct: float = 0.10
    target_pct: float    = 0.10

    # Portfolio
    apply_fees: bool    = True
    initial_capital: float = 500_000.0
    max_concurrent_positions: int = 5
    deploy_cash_frac: float = 0.25

    # Execution
    entry_on_next_open: bool = True
    exit_on_next_open: bool = True

    # Candidate ranking & filters
    benchmark_try: Tuple[str,...] = ("^CRSLDX","^CNX500","^NSE500","^NIFTY500","^BSE500","^NSEI")
    volar_lookback: int = 252
    filter_52w_window: int = 252
    within_pct_of_52w_high: float = 0.50
    top_k_daily: int = 300

    # Optional liquidity guards
    enable_basic_liquidity: bool = False
    min_price_inr: float = 50.0
    min_avg_vol_20d: float = 50_000.0

CFG = Config()

# =========================
# FEES (per user spec)
# =========================
APPLY_FEES = True

def calc_fees(turnover_buy: float, turnover_sell: float) -> float:
    if not APPLY_FEES:
        return 0.0
    BROKER_PCT = 0.001
    BROKER_MIN = 5.0
    BROKER_CAP = 20.0
    STT_PCT = 0.001
    STAMP_BUY_PCT = 0.00015
    EXCH_PCT = 0.0000297
    SEBI_PCT = 0.000001
    IPFT_PCT = 0.000001
    GST_PCT = 0.18
    DP_SELL = 20.0 if turnover_sell >= 100 else 0.0

    def _broker(turnover):
        if turnover <= 0:
            return 0.0
        fee = turnover * BROKER_PCT
        fee = max(BROKER_MIN, min(fee, BROKER_CAP))
        return fee

    br_buy  = _broker(turnover_buy)
    br_sell = _broker(turnover_sell)
    stt   = STT_PCT * (turnover_buy + turnover_sell)
    stamp = STAMP_BUY_PCT * turnover_buy
    exch  = EXCH_PCT * (turnover_buy + turnover_sell)
    sebi  = SEBI_PCT * (turnover_buy + turnover_sell)
    ipft  = IPFT_PCT * (turnover_buy + turnover_sell)
    dp    = DP_SELL
    gst_base = br_buy + br_sell + dp + exch + sebi + ipft
    gst   = GST_PCT * gst_base
    return float((br_buy + br_sell) + stt + stamp + exch + sebi + ipft + dp + gst)

# =========================
# Helpers
# =========================
def ensure_dirs(*paths):
    for p in paths:
        os.makedirs(p, exist_ok=True)

def today_str():
    return pd.Timestamp.today(tz="Asia/Kolkata").strftime("%Y-%m-%d")

def load_static_symbols(static_symbols: Optional[List[str]], static_symbols_path: Optional[str]) -> List[str]:
    syms: List[str] = []
    if static_symbols and len(static_symbols) > 0:
        syms = list(static_symbols)
    elif static_symbols_path and os.path.exists(static_symbols_path):
        with open(static_symbols_path, "r") as f:
            syms = [line.strip() for line in f if line.strip()]
    else:
        raise ValueError(
            "Provide CFG.static_symbols=[...] ('.NS' suffixes) or set CFG.static_symbols_path "
            "to a file containing one symbol per line."
        )
    out = []
    for s in syms:
        s = s.strip().upper()
        if not s.endswith(".NS"):
            s = f"{s}.NS"
        out.append(s)
    seen = set()
    uniq = []
    for s in out:
        if s not in seen:
            uniq.append(s)
            seen.add(s)
    return uniq

def fetch_prices(tickers: List[str], start: str, end: Optional[str], cache_dir: str) -> Dict[str, pd.DataFrame]:
    ensure_dirs(cache_dir)
    data = {}
    end = end or today_str()
    for ticker in tickers:
        cache_path = os.path.join(cache_dir, f"{ticker.replace('^', '_')}.parquet")
        if os.path.exists(cache_path):
            try:
                df = pd.read_parquet(cache_path)
                if len(df) and pd.to_datetime(df.index[-1]).strftime("%Y-%m-%d") >= end:
                    data[ticker] = df
                    continue
            except Exception:
                pass
        try:
            df = yf.download(ticker, start=start, end=end, auto_adjust=True, progress=False, multi_level_index=False)
            if df is None or df.empty:
                continue
            df = df.rename(columns=str.title)  # Open, High, Low, Close, Volume
            df = df[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()
            df.index.name = "date"
            df.to_parquet(cache_path)
            data[ticker] = df
        except Exception:
            continue
    return data

# =========================
# Indicators
# =========================
def cci(high: pd.Series, low: pd.Series, close: pd.Series, length: int = 20) -> pd.Series:
    """CCI = (TP - SMA(TP,n)) / (0.015 * MeanDeviation(TP,n)), TP=(H+L+C)/3"""
    tp = (high + low + close) / 3.0
    sma_tp = tp.rolling(length).mean()
    md = (tp - sma_tp).abs().rolling(length).mean()
    denom = 0.015 * md
    out = (tp - sma_tp) / denom.replace(0.0, np.nan)
    return out.replace([np.inf, -np.inf], np.nan).fillna(0.0)

def sma(series: pd.Series, length: int) -> pd.Series:
    return series.rolling(length).mean()

def ema(series: pd.Series, length: int) -> pd.Series:
    return series.ewm(span=length, adjust=False, min_periods=length).mean()

def compute_indicators(df: pd.DataFrame, cfg: Config) -> pd.DataFrame:
    out = df.copy()
    # Core
    out["cci"] = cci(out["High"], out["Low"], out["Close"], cfg.cci_len)
    out["cci_prev"] = out["cci"].shift(1)
    # NEW: EMAs & SMA trend
    out["ema_fast"] = ema(out["Close"], cfg.ema_fast)
    out["ema_slow"] = ema(out["Close"], cfg.ema_slow)
    out["sma_trend"] = sma(out["Close"], cfg.sma_trend_len)
    # Misc
    out["avg_vol_20"] = out["Volume"].rolling(20).mean()
    out["high_52w"] = out["Close"].rolling(cfg.filter_52w_window).max()
    return out.dropna()

def basic_liquidity_ok(row: pd.Series, cfg: Config) -> bool:
    if not cfg.enable_basic_liquidity:
        return True
    if row["Close"] < cfg.min_price_inr:
        return False
    if row["avg_vol_20"] < cfg.min_avg_vol_20d:
        return False
    return True

# =========================
# Per-ticker simulation
# =========================
def simulate_ticker(ticker: str, df: pd.DataFrame, cfg: Config):
    d = compute_indicators(df, cfg).copy()
    cols = ["ticker","side","date","price","shares","reason","signal_reason","score",
            "cci","ema_fast","ema_slow","sma_trend","close","high_52w"]
    if d.empty:
        return pd.DataFrame(columns=cols), pd.Series(dtype=float)

    # ---- ENTRY ----
    base_entry = (d["cci_prev"] < cfg.cci_buy_level) & (d["cci"] >= cfg.cci_buy_level)
    ema_ok = (~cfg.use_ema_cross_filter) | (d["ema_fast"] > d["ema_slow"])
    sma_ok = (~cfg.use_sma_trend_filter) | (d["Close"] > d["sma_trend"])
    entry_signal = base_entry & ema_ok & sma_ok

    # ---- INDICATOR EXITS ----
    cci_exit = (d["cci_prev"] > cfg.cci_sell_level) & (d["cci"] <= cfg.cci_sell_level)
    ema_down = cfg.exit_on_ema_cross_down & (d["ema_fast"] < d["ema_slow"])
    sma_breach = cfg.exit_on_sma_trend_breach & (d["Close"] < d["sma_trend"])
    indicator_exit = cci_exit | ema_down | sma_breach

    in_pos = False
    entry_px = stop_px = tgt_px = 0.0
    trades = []

    idx = list(d.index)
    for i in range(len(idx)-1):
        dt, nxt = idx[i], idx[i+1]
        row, nxt_row = d.loc[dt], d.loc[nxt]

        if not in_pos:
            if entry_signal.loc[dt] and basic_liquidity_ok(row, cfg):
                gap = (nxt_row["Open"] - row["Close"]) / row["Close"]
                if abs(gap) <= 0.02:
                    px = float(nxt_row["Open"] if cfg.entry_on_next_open else row["Close"])
                    parts = [f"CCI↑{cfg.cci_buy_level:.0f}"]
                    if cfg.use_ema_cross_filter: parts.append(f"EMA{cfg.ema_fast}>{cfg.ema_slow}")
                    if cfg.use_sma_trend_filter: parts.append(f"Close>SMA{cfg.sma_trend_len}")
                    sig_reason = " + ".join(parts) if parts else "CCI"
                    trades.append({
                        "ticker": ticker, "side": "BUY", "date": nxt if cfg.entry_on_next_open else dt,
                        "price": px, "shares": 0, "reason": "candidate",
                        "signal_reason": sig_reason,
                        "score": float(row["cci"]),  # simple ranking proxy
                        "cci": float(row["cci"]),
                        "ema_fast": float(row["ema_fast"]), "ema_slow": float(row["ema_slow"]),
                        "sma_trend": float(row["sma_trend"]),
                        "close": float(row["Close"]), "high_52w": float(row["high_52w"])
                    })
                    in_pos = True
                    entry_px = px
                    stop_px = entry_px * (1 - cfg.stop_loss_pct)
                    tgt_px  = entry_px * (1 + cfg.target_pct)
        else:
            hit = None
            exec_date = nxt if cfg.exit_on_next_open else dt
            # Hard exits first
            if nxt_row["Low"] <= stop_px and nxt_row["High"] >= tgt_px:
                hit, exec_price = "target", float(tgt_px)
            elif nxt_row["Low"] <= stop_px:
                hit, exec_price = "stop", float(stop_px)
            elif nxt_row["High"] >= tgt_px:
                hit, exec_price = "target", float(tgt_px)
            elif indicator_exit.loc[dt]:
                exec_price = float(nxt_row["Open"] if cfg.exit_on_next_open else row["Close"])
                reason = "indicator_exit"
                if cci_exit.loc[dt]: reason += ": CCI"
                elif ema_down.loc[dt]: reason += f": EMA{cfg.ema_fast}<{cfg.ema_slow}"
                elif sma_breach.loc[dt]: reason += f": SMA{cfg.sma_trend_len}"
                hit = reason

            if hit is not None:
                trades.append({
                    "ticker": ticker, "side": "SELL", "date": exec_date,
                    "price": float(exec_price), "shares": 0, "reason": hit,
                    "signal_reason": "", "score": np.nan,
                    "cci": float(row["cci"]),
                    "ema_fast": float(row["ema_fast"]), "ema_slow": float(row["ema_slow"]),
                    "sma_trend": float(row["sma_trend"]),
                    "close": float(row["Close"]), "high_52w": float(row["high_52w"])
                })
                in_pos = False
                entry_px = stop_px = tgt_px = 0.0

    if in_pos:
        last_dt = d.index[-1]; row = d.loc[last_dt]
        trades.append({
            "ticker": ticker, "side": "SELL", "date": last_dt,
            "price": float(row["Close"]), "shares": 0, "reason": "final_close",
            "signal_reason": "", "score": np.nan,
            "cci": float(row["cci"]),
            "ema_fast": float(row["ema_fast"]), "ema_slow": float(row["ema_slow"]),
            "sma_trend": float(row["sma_trend"]),
            "close": float(row["Close"]), "high_52w": float(row["high_52w"])
        })

    return pd.DataFrame(trades, columns=cols), pd.Series(dtype=float)

# =========================
# Benchmark, ranking & portfolio (unchanged)
# =========================
def pick_benchmark(benchmarks: Tuple[str,...], start: str, end: Optional[str], cache_dir: str) -> Tuple[str, pd.DataFrame]:
    for t in benchmarks:
        data = fetch_prices([t], start, end, cache_dir)
        df = data.get(t)
        if df is not None and not df.empty:
            log.info("Using benchmark: %s", t)
            return t, df
    idx = pd.date_range(start=start, end=end or today_str(), freq="B")
    df = pd.DataFrame({"Close": np.ones(len(idx))}, index=idx)
    log.warning("No benchmark found; using synthetic flat series.")
    return "SYNTH_BENCH", df

def compute_volar_scores(end_dt: pd.Timestamp, tickers: List[str], data_map: Dict[str,pd.DataFrame], bench_df: pd.DataFrame, lookback: int) -> Dict[str, float]:
    scores = {}
    bser = bench_df["Close"].loc[:end_dt].pct_change().dropna().iloc[-lookback:]
    for t in tickers:
        df = data_map.get(t)
        if df is None or df.empty:
            scores[t] = 0.0
            continue
        if end_dt not in df.index:
            df = df[df.index <= end_dt]
            if df.empty:
                scores[t] = 0.0
                continue
        r = df["Close"].loc[:end_dt].pct_change().dropna().iloc[-lookback:]
        common = pd.concat([r, bser], axis=1, keys=["s","b"]).dropna()
        if common.shape[0] < max(20, int(0.4*lookback)):
            scores[t] = 0.0
            continue
        excess = common["s"] - common["b"]
        vol = common["s"].std(ddof=0)
        scores[t] = 0.0 if vol <= 1e-8 else float((excess.mean() / vol) * math.sqrt(252.0))
    return scores

def markowitz_long_only(mu: np.ndarray, Sigma: np.ndarray) -> np.ndarray:
    n = len(mu)
    eps = 1e-6
    Sigma = Sigma + eps*np.eye(n)

    def solve_lambda(lmbd: float, active_mask=None):
        if active_mask is None:
            A = np.block([[2*lmbd*Sigma, np.ones((n,1))],[np.ones((1,n)), np.zeros((1,1))]])
            b = np.concatenate([mu, np.array([1.0])])
            try:
                sol = np.linalg.solve(A, b)
                w = sol[:n]
            except np.linalg.LinAlgError:
                w = np.full(n, 1.0/n)
            return w
        else:
            idx = np.where(active_mask)[0]
            if len(idx)==0:
                return np.full(n, 1.0/n)
            S = Sigma[np.ix_(idx, idx)]
            o = np.ones(len(idx))
            m = mu[idx]
            A = np.block([[2*lmbd*S, o[:,None]],[o[None,:], np.zeros((1,1))]])
            b = np.concatenate([m, np.array([1.0])])
            try:
                sol = np.linalg.solve(A, b)
                w_sub = sol[:len(idx)]
            except np.linalg.LinAlgError:
                w_sub = np.full(len(idx), 1.0/len(idx))
            w = np.zeros(n); w[idx] = w_sub
            return w

    best_w = np.full(n, 1.0/n)
    best_sr = -1e9
    for lmbd in np.logspace(-3, 3, 31):
        active = np.ones(n, dtype=bool)
        w = None
        for _ in range(n):
            w = solve_lambda(lmbd, active_mask=active)
            neg = w < 0
            if not neg.any():
                break
            worst = np.argmin(w)
            active[worst] = False
        if w is None:
            continue
        w = np.clip(w, 0, None)
        if w.sum() <= 0:
            continue
        w = w / w.sum()
        mu_p = float(mu @ w)
        vol_p = float(np.sqrt(w @ Sigma @ w))
        if vol_p <= 1e-8:
            continue
        sr = mu_p / vol_p
        if sr > best_sr:
            best_sr = sr
            best_w = w.copy()
    return best_w

def aggregate_and_apply(all_trades: pd.DataFrame, data_map: Dict[str, pd.DataFrame], bench_df: pd.DataFrame, cfg: Config):
    if all_trades.empty:
        return all_trades, pd.Series(dtype=float), {}

    side_order = {"BUY": 0, "SELL": 1}
    all_trades = (all_trades
        .assign(_sorder=all_trades["side"].map(side_order))
        .sort_values(by=["date", "_sorder"], kind="stable")
        .drop(columns=["_sorder"])
        .reset_index(drop=True)
    )
    all_trades["date"] = pd.to_datetime(all_trades["date"])

    equity_curve = []
    dates = sorted(all_trades["date"].unique().tolist())
    cash = cfg.initial_capital
    open_positions = {}
    completed_legs = []

    global APPLY_FEES
    APPLY_FEES = cfg.apply_fees

    def _get_close_on(tkr, dt):
        df = data_map.get(tkr)
        if df is None or df.empty:
            return np.nan
        if dt in df.index:
            return float(df.loc[dt, "Close"])
        prev = df[df.index <= dt]
        if prev.empty:
            return np.nan
        return float(prev["Close"].iloc[-1])

    if dates:
        seed_date = pd.to_datetime(dates[0]) - pd.Timedelta(days=1)
        equity_curve.append((seed_date, float(cash)))

    for dt in dates:
        day_trades = all_trades[all_trades["date"] == dt].copy()

        # ---- SELL first ----
        for _, tr in day_trades[day_trades["side"] == "SELL"].iterrows():
            tkr = tr["ticker"]
            price = float(tr["price"])
            pos = open_positions.get(tkr)
            if pos is None:
                continue
            shares = int(pos["shares"])
            turnover_sell = shares * price
            fee = calc_fees(0.0, turnover_sell)
            pnl = (price - pos["entry_px"]) * shares
            cash += (turnover_sell - fee)
            realized = pnl - fee - pos.get("buy_fee", 0.0)
            completed_legs.append({
                "ticker": tkr, "side": "SELL", "date": dt, "price": price,
                "shares": shares, "reason": tr.get("reason",""),
                "turnover": turnover_sell, "fees_inr": fee, "pnl_inr": realized,
                "cci": tr.get("cci", np.nan),
                "ema_fast": tr.get("ema_fast", np.nan), "ema_slow": tr.get("ema_slow", np.nan),
                "sma_trend": tr.get("sma_trend", np.nan),
                "close": tr.get("close", np.nan), "high_52w": tr.get("high_52w", np.nan),
                "volar": tr.get("volar", np.nan), "mvo_weight": np.nan, "alloc_inr": np.nan
            })
            del open_positions[tkr]

        # ---- BUY candidates ----
        buys_today = day_trades[day_trades["side"] == "BUY"].copy()
        # 52w filter
        if not buys_today.empty:
            keep = []
            for _, rr in buys_today.iterrows():
                df = data_map.get(rr["ticker"])
                if df is None or df.empty or dt not in df.index:
                    continue
                close = float(df.loc[dt, "Close"])
                hist = df["Close"].loc[:dt]
                window = hist.iloc[-cfg.filter_52w_window:] if len(hist)>=cfg.filter_52w_window else hist
                high_52w = float(window.max())
                if high_52w>0 and close >= cfg.within_pct_of_52w_high * high_52w:
                    keep.append(rr)
            buys_today = pd.DataFrame(keep) if keep else pd.DataFrame(columns=buys_today.columns)

        # Exclude already-held
        if not buys_today.empty:
            buys_today = buys_today[~buys_today["ticker"].isin(open_positions.keys())]

        # VOLAR ranking
        if not buys_today.empty:
            tickers = buys_today["ticker"].tolist()
            volar_scores = compute_volar_scores(dt, tickers, data_map, bench_df, cfg.volar_lookback)
            buys_today["volar"] = buys_today["ticker"].map(volar_scores)
            buys_today = buys_today.sort_values("volar", ascending=False).reset_index(drop=True)

        slots = cfg.max_concurrent_positions - len(open_positions)
        selected = pd.DataFrame(columns=buys_today.columns)
        if slots > 0 and not buys_today.empty:
            selected = buys_today.head(min(cfg.top_k_daily, slots)).copy()

        if not selected.empty:
            log.info("Selected %d BUY candidates on %s:", selected.shape[0], dt.date())
            for i, rr in selected.reset_index(drop=True).iterrows():
                log.info("  %-12s volar=%6.2f rank=%d px=%8.2f", rr["ticker"], rr.get("volar",0.0), i+1, rr["price"])

            # MVO sizing
            names = selected["ticker"].tolist()
            rets = []
            for t in names:
                df = data_map.get(t)
                ser = df["Close"].loc[:dt].pct_change().dropna().iloc[-cfg.volar_lookback:]
                rets.append(ser)
            R = pd.concat(rets, axis=1)
            R.columns = names
            R = R.dropna()
            if R.empty or R.shape[0] < max(20, int(0.4*cfg.volar_lookback)) or R.shape[1] == 0:
                weights = np.full(len(names), 1.0/len(names))
            else:
                mu = R.mean().values
                Sigma = R.cov().values
                weights = markowitz_long_only(mu, Sigma)

            # Cap daily deployed cash
            deploy_cash = max(0.0, float(cash)) * float(cfg.deploy_cash_frac)

            if deploy_cash <= 0:
                log.info("No deployable cash (cap=%.0f%%) on %s", 100*cfg.deploy_cash_frac, dt.date())
            else:
                alloc = (weights / weights.sum()) * deploy_cash if weights.sum()>0 else np.full(len(names), deploy_cash/len(names))
                # Pre-compute ranks for reason strings
                rank_map = {row["ticker"]: (idx+1) for idx, (_, row) in enumerate(selected.iterrows())}
                for w_amt, t in zip(alloc, names):
                    df_t = data_map[t]
                    price = float(df_t.loc[dt, "Close"] if dt in df_t.index else df_t["Close"].loc[:dt].iloc[-1])
                    shares = int(math.floor(w_amt / price))
                    if shares <= 0:
                        log.info("Skip BUY %-12s (alloc %.2f too small)", t, w_amt)
                        continue
                    turn = shares * price
                    fee = calc_fees(turn, 0.0)
                    total_cost = turn + fee
                    if total_cost > cash:
                        shares = int(math.floor((cash - fee) / price))
                        if shares <= 0:
                            log.info("Skip BUY %-12s due to cash/fees", t)
                            continue
                        turn = shares * price
                        total_cost = turn + fee
                    cash -= total_cost
                    open_positions[t] = {"entry_date": dt, "entry_px": price, "shares": shares, "buy_fee": fee, "entry_reason": "entry"}
                    # Reason string
                    row_sel = selected[selected["ticker"]==t].iloc[0]
                    volar_val = float(row_sel.get("volar", np.nan))
                    rank_pos = rank_map.get(t, np.nan)
                    high_52w = float(row_sel.get("high_52w", np.nan))
                    close_val = float(row_sel.get("close", np.nan))
                    pct_52w = (close_val / high_52w) if (high_52w and high_52w>0) else np.nan
                    mvo_weight_today = (w_amt / deploy_cash) if deploy_cash > 0 else 0.0
                    sig_reason = row_sel.get("signal_reason", f"CCI↑{CFG.cci_buy_level:.0f}")
                    reason_text = (
                        f"{sig_reason}; 52w%={pct_52w:.1%} (>= {CFG.within_pct_of_52w_high:.0%}); "
                        f"VOLAR rank {int(rank_pos)}/{len(names)} (VOLAR={volar_val:.2f}); "
                        f"MVO weight={mvo_weight_today:.1%} of capped cash ({100*CFG.deploy_cash_frac:.0f}% of available)"
                    )
                    completed_legs.append({
                        "ticker": t, "side": "BUY", "date": dt, "price": price,
                        "shares": shares, "reason": reason_text,
                        "turnover": turn, "fees_inr": fee, "pnl_inr": 0.0,
                        "cci": float(row_sel.get("cci", np.nan)),
                        "ema_fast": float(row_sel.get("ema_fast", np.nan)),
                        "ema_slow": float(row_sel.get("ema_slow", np.nan)),
                        "sma_trend": float(row_sel.get("sma_trend", np.nan)),
                        "close": close_val, "high_52w": high_52w,
                        "volar": volar_val, "mvo_weight": float(mvo_weight_today), "alloc_inr": float(w_amt)
                    })
                    log.info("BUY %-12s px=%8.2f sh=%6d fee=%.2f cash=%.2f :: %s",
                             t, price, shares, fee, cash, reason_text)

        # MTM valuation
        mtm = 0.0
        for _tkr, pos in open_positions.items():
            px = _get_close_on(_tkr, dt)
            if not np.isnan(px):
                mtm += pos["shares"] * px
        total_equity = cash + mtm
        equity_curve.append((dt, float(total_equity)))

    eq_ser = pd.Series([e for _, e in equity_curve], index=[d for d, _ in equity_curve])
    legs_df = pd.DataFrame(completed_legs).sort_values(["date", "ticker", "side"]).reset_index(drop=True)

    # Roundtrips
    roundtrips = []
    by_tkr_open = {}
    for _, leg in legs_df.iterrows():
        tkr = leg["ticker"]
        if leg["side"] == "BUY":
            by_tkr_open[tkr] = leg
        else:
            buy = by_tkr_open.pop(tkr, None)
            if buy is None:
                continue
            fees_total = float(buy.get("fees_inr", 0.0) + leg.get("fees_inr", 0.0))
            gross_pnl = (leg["price"] - buy["price"]) * buy["shares"]
            net_pnl   = gross_pnl - fees_total
            ret_pct   = (leg["price"] / buy["price"] - 1.0) * 100.0
            days_held = (pd.to_datetime(leg["date"]) - pd.to_datetime(buy["date"])).days
            roundtrips.append({
                "ticker": tkr,
                "entry_date": pd.to_datetime(buy["date"]),
                "entry_price": float(buy["price"]),
                "exit_date": pd.to_datetime(leg["date"]),
                "exit_price": float(leg["price"]),
                "days_held": int(days_held),
                "shares": int(buy["shares"]),
                "entry_reason": buy.get("reason",""),
                "exit_reason": leg.get("reason",""),
                "gross_pnl_inr": float(gross_pnl),
                "fees_total_inr": float(fees_total),
                "net_pnl_inr": float(net_pnl),
                "return_pct": float(ret_pct),
                "cci_entry": float(buy.get("cci", np.nan)),
                "ema_fast_entry": float(buy.get("ema_fast", np.nan)),
                "ema_slow_entry": float(buy.get("ema_slow", np.nan)),
                "sma_trend_entry": float(buy.get("sma_trend", np.nan)),
                "close_entry": float(buy.get("close", np.nan)),
                "high_52w_entry": float(buy.get("high_52w", np.nan)),
                "volar_entry": float(buy.get("volar", np.nan)),
                "mvo_weight_entry": float(buy.get("mvo_weight", np.nan)),
                "alloc_inr_entry": float(buy.get("alloc_inr", np.nan))
            })
    trips_df = pd.DataFrame(roundtrips).sort_values(["entry_date","ticker"]).reset_index(drop=True)

    metrics = compute_metrics(eq_ser, legs_df)
    return legs_df, trips_df, eq_ser, metrics

# =========================
# Metrics & plotting
# =========================
def compute_metrics(equity: pd.Series, legs_df: pd.DataFrame):
    out = {}
    if equity is None or equity.empty:
        return out
    eq = equity.dropna()
    daily_ret = eq.pct_change().fillna(0.0)

    days = (eq.index[-1] - eq.index[0]).days or 1
    years = days / 365.25
    cagr = (eq.iloc[-1] / eq.iloc[0]) ** (1/years) - 1 if years > 0 else 0.0
    sharpe = (daily_ret.mean() / daily_ret.std(ddof=0) * np.sqrt(252)) if daily_ret.std(ddof=0) > 0 else 0.0

    cummax = eq.cummax()
    dd = (eq - cummax) / cummax
    max_dd = dd.min()

    wins = 0
    n_sells = legs_df[legs_df["side"] == "SELL"].shape[0] if legs_df is not None and not legs_df.empty else 0
    for _, r in legs_df[legs_df["side"] == "SELL"].iterrows():
        if float(r.get("pnl_inr", 0.0)) > 0:
            wins += 1
    win_rate = (wins / n_sells) * 100.0 if n_sells > 0 else 0.0

    out.update({
        "start_equity_inr": float(eq.iloc[0]),
        "final_equity_inr": float(eq.iloc[-1]),
        "cagr_pct": float(cagr * 100),
        "sharpe": float(sharpe),
        "max_drawdown_pct": float(max_dd * 100),
        "win_rate_pct": float(win_rate),
        "n_trades": int(n_sells),
    })
    return out

def plot_equity(equity: pd.Series, out_path: str):
    if equity is None or equity.empty:
        return
    try:
        import matplotlib.pyplot as plt
        plt.figure(figsize=(10,5))
        plt.plot(equity.index, equity.values)
        plt.title("Equity Curve")
        plt.xlabel("Date")
        plt.ylabel("Equity (INR)")
        plt.tight_layout()
        plt.savefig(out_path)
        plt.close()
    except Exception:
        pass

# =========================
# Orchestration
# =========================
def backtest(cfg: Config):
    ensure_dirs(cfg.cache_dir, cfg.out_dir)
    log.info("Universe: loading static symbols...")
    symbols = load_static_symbols(cfg.static_symbols, cfg.static_symbols_path)
    log.info("Loaded %d symbols.", len(symbols))

    log.info("Data: fetching OHLCV from yfinance (adjusted)...")
    data_map = fetch_prices(symbols, cfg.start_date, cfg.end_date, cfg.cache_dir)
    log.info("Downloaded %d symbols with data.", len(data_map))

    bench_tkr, bench_df = pick_benchmark(cfg.benchmark_try, cfg.start_date, cfg.end_date, cfg.cache_dir)
    log.info("Benchmark selected: %s", bench_tkr)

    log.info("Signals: CCI=%d | EMA(%d/%d) %s | SMA(%d) %s",
             cfg.cci_len, cfg.ema_fast, cfg.ema_slow,
             "ON" if cfg.use_ema_cross_filter else "OFF",
             cfg.sma_trend_len, "ON" if cfg.use_sma_trend_filter else "OFF")

    all_trades = []
    for i, tkr in enumerate(symbols, 1):
        df = data_map.get(tkr)
        if df is None or df.empty:
            continue
        tr, _ = simulate_ticker(tkr, df, cfg)
        if not tr.empty:
            all_trades.append(tr)
        if i % 50 == 0:
            log.info("  processed %d/%d tickers...", i, len(symbols))

    if not all_trades:
        log.warning("No signals. Try toggling filters (use_ema_cross_filter/use_sma_trend_filter=False) "
                    "or relaxing CCI (buy_level=-150, cci_len=14) or 52w filter.")
        return None, None, None, {}
    all_trades = pd.concat(all_trades, ignore_index=True)

    log.info("Portfolio: cap daily deploy to %.0f%% of cash; 52w>=%.0f%% high; top-%d by VOLAᵣ; MVO; max %d positions.",
             cfg.deploy_cash_frac*100, cfg.within_pct_of_52w_high*100, cfg.top_k_daily, cfg.max_concurrent_positions)
    legs_df, trips_df, equity, metrics = aggregate_and_apply(all_trades, data_map, bench_df, cfg)

    stamp = pd.Timestamp.today(tz="Asia/Kolkata").strftime("%Y%m%d_%H%M%S")
    legs_path = os.path.join(cfg.out_dir, f"trades_legs_{stamp}.csv")
    trips_path = os.path.join(cfg.out_dir, f"trades_roundtrips_{stamp}.csv")
    equity_path = os.path.join(cfg.out_dir, f"equity_{stamp}.csv")
    metrics_path = os.path.join(cfg.out_dir, f"metrics_{stamp}.json")
    eq_plot_path = os.path.join(cfg.out_dir, f"equity_{stamp}.png")

    if legs_df is not None:
        legs_df.to_csv(legs_path, index=False)
    if trips_df is not None:
        trips_df.to_csv(trips_path, index=False)
    if equity is not None:
        pd.DataFrame({"date": equity.index, "equity": equity.values}).to_csv(equity_path, index=False)
    with open(metrics_path, "w") as f:
        json.dump(metrics, f, indent=2)
    if cfg.plot and equity is not None:
        plot_equity(equity, eq_plot_path)

    log.info("=== METRICS ===\n%s", json.dumps(metrics, indent=2))
    log.info("Files written:\n  %s\n  %s\n  %s\n  %s", legs_path, trips_path, equity_path, metrics_path)
    if cfg.plot:
        log.info("  %s", eq_plot_path)

def main():
    global APPLY_FEES
    APPLY_FEES = bool(CFG.apply_fees)
    # Universe: set directly or via CFG.static_symbols_path
    # CFG.static_symbols = ['360ONE.NS', '3MINDIA.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABB.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABLBL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'ACC.NS', 'ACE.NS', 'ACMESOLAR.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'AEGISLOG.NS', 'AEGISVOPAK.NS', 'AFCONS.NS', 'AFFLE.NS', 'AGARWALEYE.NS', 'AIAENG.NS', 'AIIL.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'AKZOINDIA.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APLAPOLLO.NS', 'APLLTD.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ARE&M.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAL.NS', 'ASTRAZEN.NS', 'ATGL.NS', 'ATHERENERG.NS', 'ATUL.NS', 'AUBANK.NS', 'AUROPHARMA.NS', 'AWL.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJAJFINSV.NS', 'BAJAJHFL.NS', 'BAJAJHLDNG.NS', 'BAJFINANCE.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'BASF.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BBTC.NS', 'BDL.NS', 'BEL.NS', 'BEML.NS', 'BERGEPAINT.NS', 'BHARATFORG.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BHEL.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BLS.NS', 'BLUEDART.NS', 'BLUEJET.NS', 'BLUESTARCO.NS', 'BOSCHLTD.NS', 'BPCL.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'BSE.NS', 'BSOFT.NS', 'CAMPUS.NS', 'CAMS.NS', 'CANBK.NS', 'CANFINHOME.NS', 'CAPLIPOINT.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CCL.NS', 'CDSL.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CESC.NS', 'CGCL.NS', 'CGPOWER.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOICEIN.NS', 'CHOLAFIN.NS', 'CHOLAHLDNG.NS', 'CIPLA.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CONCOR.NS', 'CONCORDBIO.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CRISIL.NS', 'CROMPTON.NS', 'CUB.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DBREALTY.NS', 'DCMSHRIRAM.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'DLF.NS', 'DMART.NS', 'DOMS.NS', 'DRREDDY.NS', 'ECLERX.NS', 'EICHERMOT.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ENRIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'FACT.NS', 'FEDERALBNK.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FIRSTCRY.NS', 'FIVESTAR.NS', 'FLUOROCHEM.NS', 'FORCEMOT.NS', 'FORTIS.NS', 'FSL.NS', 'GAIL.NS', 'GESHIP.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'GMDCLTD.NS', 'GMRAIRPORT.NS', 'GODFRYPHLP.NS', 'GODIGIT.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GPIL.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GRSE.NS', 'GSPL.NS', 'GUJGASLTD.NS', 'GVT&D.NS', 'HAL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEG.NS', 'HEROMOTOCO.NS', 'HEXT.NS', 'HFCL.NS', 'HINDALCO.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HSCL.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDEA.NS', 'IDFCFIRSTB.NS', 'IEX.NS', 'IFCI.NS', 'IGIL.NS', 'IGL.NS', 'IIFL.NS', 'IKS.NS', 'INDGN.NS', 'INDHOTEL.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'INDIGO.NS', 'INDUSINDBK.NS', 'INDUSTOWER.NS', 'INFY.NS', 'INOXINDIA.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'IOB.NS', 'IOC.NS', 'IPCALAB.NS', 'IRB.NS', 'IRCON.NS', 'IRCTC.NS', 'IREDA.NS', 'IRFC.NS', 'ITC.NS', 'ITCHOTELS.NS', 'ITI.NS', 'J&KBANK.NS', 'JBCHEPHARM.NS', 'JBMA.NS', 'JINDALSAW.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JKCEMENT.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JPPOWER.NS', 'JSL.NS', 'JSWENERGY.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JWL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KAJARIACER.NS', 'KALYANKJIL.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KEI.NS', 'KFINTECH.NS', 'KIMS.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KOTAKBANK.NS', 'KPIL.NS', 'KPITTECH.NS', 'KPRMILL.NS', 'KSB.NS', 'LALPATHLAB.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICHSGFIN.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LODHA.NS', 'LT.NS', 'LTF.NS', 'LTFOODS.NS', 'LTIM.NS', 'LTTS.NS', 'LUPIN.NS', 'M&M.NS', 'M&MFIN.NS', 'MAHABANK.NS', 'MAHSCOOTER.NS', 'MAHSEAMLES.NS', 'MANAPPURAM.NS', 'MANKIND.NS', 'MANYAVAR.NS', 'MAPMYINDIA.NS', 'MARICO.NS', 'MARUTI.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MCX.NS', 'MEDANTA.NS', 'METROPOLIS.NS', 'MFSL.NS', 'MGL.NS', 'MINDACORP.NS', 'MMTC.NS', 'MOTHERSON.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MRF.NS', 'MRPL.NS', 'MSUMI.NS', 'MUTHOOTFIN.NS', 'NAM-INDIA.NS', 'NATCOPHARM.NS', 'NATIONALUM.NS', 'NAUKRI.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NBCC.NS', 'NCC.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NH.NS', 'NHPC.NS', 'NIACL.NS', 'NIVABUPA.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPC.NS', 'NTPCGREEN.NS', 'NUVAMA.NS', 'NUVOCO.NS', 'NYKAA.NS', 'OBEROIRLTY.NS', 'OFSS.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'ONESOURCE.NS', 'ONGC.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PAYTM.NS', 'PCBL.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFC.NS', 'PFIZER.NS', 'PGEL.NS', 'PGHH.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PIIND.NS', 'PNB.NS', 'PNBHOUSING.NS', 'POLICYBZR.NS', 'POLYCAB.NS', 'POLYMED.NS', 'POONAWALLA.NS', 'POWERGRID.NS', 'POWERINDIA.NS', 'PPLPHARMA.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PTCIL.NS', 'PVRINOX.NS', 'RADICO.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RAMCOCEM.NS', 'RBLBANK.NS', 'RCF.NS', 'RECLTD.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RELINFRA.NS', 'RHIM.NS', 'RITES.NS', 'RKFORGE.NS', 'RPOWER.NS', 'RRKABEL.NS', 'RVNL.NS', 'SAGILITY.NS', 'SAIL.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SBIN.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'SRF.NS', 'STARHEALTH.NS', 'SUMICHEM.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANCORP.NS', 'SWIGGY.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TARIL.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TBOTEK.NS', 'TCS.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'THELEELA.NS', 'THERMAX.NS', 'TIINDIA.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRITURBINE.NS', 'TRIVENI.NS', 'TTML.NS', 'TVSMOTOR.NS', 'UBL.NS', 'UCOBANK.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UNITDSPR.NS', 'UNOMINDA.NS', 'UPL.NS', 'USHAMART.NS', 'UTIAMC.NS', 'VBL.NS', 'VEDL.NS', 'VENTIVE.NS', 'VGUARD.NS', 'VIJAYA.NS', 'VMM.NS', 'VOLTAS.NS', 'VTL.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZEEL.NS', 'ZENSARTECH.NS', 'ZENTEC.NS', 'ZFCVINDIA.NS', 'ZYDUSLIFE.NS']
    CFG.static_symbols_path = "nifty500.txt"
    backtest(CFG)

if __name__ == "__main__":
    main()


2025-10-13 14:38:04 | INFO | Universe: loading static symbols...
2025-10-13 14:38:04 | INFO | Loaded 500 symbols.
2025-10-13 14:38:04 | INFO | Data: fetching OHLCV from yfinance (adjusted)...
2025-10-13 14:38:08 | ERROR | 
1 Failed download:
2025-10-13 14:38:08 | ERROR | ['ABLBL.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2025-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1420050600, endDate = 1735669800")')
2025-10-13 14:38:13 | ERROR | 
1 Failed download:
2025-10-13 14:38:13 | ERROR | ['AEGISVOPAK.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2025-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1420050600, endDate = 1735669800")')
2025-10-13 14:38:14 | ERROR | 
1 Failed download:
2025-10-13 14:38:14 | ERROR | ['AGARWALEYE.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2025-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1420050600

In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
CCI + Williams %R Pullback-in-Uptrend (NSE) — with per-rule ON/OFF toggles

LONG SETUP (default ON)
-----------------------
Trend Filter (toggle: use_trend_filter_ema50):
    • Price > EMA(50)

Oversold Pullback (toggles: use_wpr_oversold, use_cci_oversold):
    • Williams %R < -80
    • CCI < -100

Entry Confirmation (toggles: require_wpr_confirm_above_m50, require_cci_confirm_above_m100):
    • W%R crosses up above -50 after being below -80
    • CCI crosses up above -100 after being below -100

EXITS (first hit; each toggleable)
----------------------------------
• Hard stop / take-profit
• Momentum fade:
    - W%R overbought (> -20) then falls back below -50 (exit_on_wpr_overbought_then_below_m50)
    - CCI > +100 then falls back below +100 (exit_on_cci_fall_below_p100)
• Trend break:
    - Close < EMA(50) (exit_on_trend_break_ema50)
    - Break recent swing-low (rolling N-bar Low) (exit_on_swing_low_break, swing_lookback_n)

Notes
-----
• EMA-fast/slow & SMA-trend from your prior version are kept and still toggleable as *extra filters/exits* if you want them.
• Portfolio layer preserved: 52w filter, VOLAR ranking, long-only MVO sizing, cash cap, fees, metrics, CSVs, equity PNG.
"""

import os, json, math, warnings, logging
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore", category=FutureWarning)

try:
    import yfinance as yf
except Exception:
    pass

# =========================
# LOGGING
# =========================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
log = logging.getLogger("cci_wpr_pullback_uptrend_v1")

# =========================
# CONFIG
# =========================
@dataclass
class Config:
    # Data
    start_date: str = "2015-01-01"
    end_date: str   = "2025-01-01"
    static_symbols: Optional[List[str]] = None
    static_symbols_path: Optional[str] = None
    cache_dir: str = "cache"
    out_dir: str   = "outputs"
    plot: bool     = True

    # --- Core indicator params ---
    cci_len: int = 20
    wpr_len: int = 14               # Williams %R lookback
    ema_trend_len: int = 50         # Trend filter EMA(50)

    # Thresholds
    wpr_oversold: float = -80.0
    wpr_confirm: float  = -50.0
    cci_oversold: float = -100.0
    cci_momentum_hi: float = +100.0

    # --- Toggles: ENTRY RULES (all True by default) ---
    use_trend_filter_ema50: bool = True              # require Close > EMA(50)
    use_wpr_oversold: bool = True                    # use W%R < -80 as part of pullback
    use_cci_oversold: bool = True                    # use CCI < -100 as part of pullback
    require_wpr_confirm_above_m50: bool = True       # require W%R cross up above -50 (after < -80)
    require_cci_confirm_above_m100: bool = True      # require CCI cross up above -100 (after < -100)

    # --- Optional extra filters from earlier version (can keep OFF) ---
    ema_fast: int = 10
    ema_slow: int = 20
    sma_trend_len: int = 200
    use_ema_cross_filter: bool = False               # require EMA_fast > EMA_slow at entry
    use_sma_trend_filter: bool = False               # require Close > SMA(200) at entry

    # --- Toggles: EXIT RULES ---
    exit_on_wpr_overbought_then_below_m50: bool = True   # W%R > -20 then <= -50
    exit_on_cci_fall_below_p100: bool = True             # CCI > +100 then <= +100
    exit_on_trend_break_ema50: bool = True               # Close < EMA(50)
    exit_on_swing_low_break: bool = True                 # Low < rolling N-bar low
    swing_lookback_n: int = 10

    # Also keep these optional exits if desired:
    exit_on_ema_cross_down: bool = False                 # EMA_fast < EMA_slow
    exit_on_sma_trend_breach: bool = False               # Close < SMA(200)

    # Fixed exits
    stop_loss_pct: float = 0.10
    target_pct: float    = 0.10

    # Portfolio
    apply_fees: bool    = True
    initial_capital: float = 500_000.0
    max_concurrent_positions: int = 5
    deploy_cash_frac: float = 0.25

    # Execution
    entry_on_next_open: bool = True
    exit_on_next_open: bool = True

    # Candidate ranking & filters
    benchmark_try: Tuple[str,...] = ("^CRSLDX","^CNX500","^NSE500","^NIFTY500","^BSE500","^NSEI")
    volar_lookback: int = 252
    filter_52w_window: int = 252
    within_pct_of_52w_high: float = 0.50
    top_k_daily: int = 300

    # Optional liquidity guards
    enable_basic_liquidity: bool = False
    min_price_inr: float = 50.0
    min_avg_vol_20d: float = 50_000.0

CFG = Config()

# =========================
# FEES
# =========================
APPLY_FEES = True
def calc_fees(turnover_buy: float, turnover_sell: float) -> float:
    if not APPLY_FEES:
        return 0.0
    BROKER_PCT = 0.001; BROKER_MIN = 5.0; BROKER_CAP = 20.0
    STT_PCT = 0.001; STAMP_BUY_PCT = 0.00015; EXCH_PCT = 0.0000297
    SEBI_PCT = 0.000001; IPFT_PCT = 0.000001; GST_PCT = 0.18
    DP_SELL = 20.0 if turnover_sell >= 100 else 0.0

    def _broker(turnover):
        if turnover <= 0: return 0.0
        fee = turnover * BROKER_PCT
        return max(BROKER_MIN, min(fee, BROKER_CAP))

    br_buy  = _broker(turnover_buy)
    br_sell = _broker(turnover_sell)
    stt   = STT_PCT * (turnover_buy + turnover_sell)
    stamp = STAMP_BUY_PCT * turnover_buy
    exch  = EXCH_PCT * (turnover_buy + turnover_sell)
    sebi  = SEBI_PCT * (turnover_buy + turnover_sell)
    ipft  = IPFT_PCT * (turnover_buy + turnover_sell)
    dp    = DP_SELL
    gst   = GST_PCT * (br_buy + br_sell + dp + exch + sebi + ipft)
    return float((br_buy + br_sell) + stt + stamp + exch + sebi + ipft + dp + gst)

# =========================
# Helpers
# =========================
def ensure_dirs(*paths):
    for p in paths: os.makedirs(p, exist_ok=True)

def today_str():
    return pd.Timestamp.today(tz="Asia/Kolkata").strftime("%Y-%m-%d")

def load_static_symbols(static_symbols: Optional[List[str]], static_symbols_path: Optional[str]) -> List[str]:
    syms: List[str] = []
    if static_symbols and len(static_symbols) > 0:
        syms = list(static_symbols)
    elif static_symbols_path and os.path.exists(static_symbols_path):
        with open(static_symbols_path, "r") as f:
            syms = [line.strip() for line in f if line.strip()]
    else:
        raise ValueError("Provide CFG.static_symbols=[...] ('.NS') or CFG.static_symbols_path to a file.")
    out = []
    for s in syms:
        s = s.strip().upper()
        if not s.endswith(".NS"): s = f"{s}.NS"
        out.append(s)
    uniq = []
    seen = set()
    for s in out:
        if s not in seen:
            uniq.append(s); seen.add(s)
    return uniq

def fetch_prices(tickers: List[str], start: str, end: Optional[str], cache_dir: str) -> Dict[str, pd.DataFrame]:
    ensure_dirs(cache_dir)
    data = {}; end = end or today_str()
    for ticker in tickers:
        cache_path = os.path.join(cache_dir, f"{ticker.replace('^', '_')}.parquet")
        if os.path.exists(cache_path):
            try:
                df = pd.read_parquet(cache_path)
                if len(df) and pd.to_datetime(df.index[-1]).strftime("%Y-%m-%d") >= end:
                    data[ticker] = df; continue
            except Exception:
                pass
        try:
            df = yf.download(ticker, start=start, end=end, auto_adjust=True, progress=False, multi_level_index=False)
            if df is None or df.empty: continue
            df = df.rename(columns=str.title)[['Open','High','Low','Close','Volume']].dropna()
            df.index.name = "date"; df.to_parquet(cache_path); data[ticker] = df
        except Exception:
            continue
    return data

# =========================
# Indicators
# =========================
def sma(series: pd.Series, length: int) -> pd.Series:
    return series.rolling(length).mean()

def ema(series: pd.Series, length: int) -> pd.Series:
    return series.ewm(span=length, adjust=False, min_periods=length).mean()

def cci(high: pd.Series, low: pd.Series, close: pd.Series, length: int = 20) -> pd.Series:
    tp = (high + low + close) / 3.0
    sma_tp = tp.rolling(length).mean()
    md = (tp - sma_tp).abs().rolling(length).mean()
    denom = (0.015 * md).replace(0.0, np.nan)
    out = (tp - sma_tp) / denom
    return out.replace([np.inf, -np.inf], np.nan).fillna(0.0)

def williams_r(high: pd.Series, low: pd.Series, close: pd.Series, length: int = 14) -> pd.Series:
    hh = high.rolling(length).max()
    ll = low.rolling(length).min()
    denom = (hh - ll).replace(0.0, np.nan)
    wpr = -100.0 * (hh - close) / denom
    return wpr.replace([np.inf, -np.inf], np.nan).fillna(-50.0)  # center default

def compute_indicators(df: pd.DataFrame, cfg: Config) -> pd.DataFrame:
    out = df.copy()
    # Trend & context
    out["ema_trend50"] = ema(out["Close"], cfg.ema_trend_len)
    out["ema_fast"]    = ema(out["Close"], cfg.ema_fast)
    out["ema_slow"]    = ema(out["Close"], cfg.ema_slow)
    out["sma_trend"]   = sma(out["Close"], cfg.sma_trend_len)
    # Oscillators
    out["cci"] = cci(out["High"], out["Low"], out["Close"], cfg.cci_len)
    out["wpr"] = williams_r(out["High"], out["Low"], out["Close"], cfg.wpr_len)
    # Previous values for cross logic
    out["cci_prev"] = out["cci"].shift(1)
    out["wpr_prev"] = out["wpr"].shift(1)
    # Swing-low line for exit
    out["swing_low"] = out["Low"].rolling(cfg.swing_lookback_n).min()
    # Misc
    out["avg_vol_20"] = out["Volume"].rolling(20).mean()
    out["high_52w"] = out["Close"].rolling(cfg.filter_52w_window).max()
    return out.dropna()

def basic_liquidity_ok(row: pd.Series, cfg: Config) -> bool:
    if not cfg.enable_basic_liquidity: return True
    if row["Close"] < cfg.min_price_inr: return False
    if row["avg_vol_20"] < cfg.min_avg_vol_20d: return False
    return True

# =========================
# Per-ticker simulation
# =========================
def simulate_ticker(ticker: str, df: pd.DataFrame, cfg: Config):
    d = compute_indicators(df, cfg).copy()
    cols = ["ticker","side","date","price","shares","reason","signal_reason","score",
            "cci","wpr","ema_trend50","ema_fast","ema_slow","sma_trend","swing_low","close","high_52w"]
    if d.empty:
        return pd.DataFrame(columns=cols), pd.Series(dtype=float)

    # ---------- ENTRY ----------
    # Trend filter: price above EMA(50)
    trend_ok = (~cfg.use_trend_filter_ema50) | (d["Close"] > d["ema_trend50"])

    # Oversold pullback (armed) — previous bar(s) must be oversold
    wpr_armed = (~cfg.use_wpr_oversold) | (d["wpr_prev"] <= cfg.wpr_oversold)
    cci_armed = (~cfg.use_cci_oversold) | (d["cci_prev"] <= cfg.cci_oversold)

    # Confirmation crosses up on current bar close:
    wpr_conf = (~cfg.require_wpr_confirm_above_m50) | ((d["wpr_prev"] <= cfg.wpr_oversold) & (d["wpr"] > cfg.wpr_confirm))
    cci_conf = (~cfg.require_cci_confirm_above_m100) | ((d["cci_prev"] < cfg.cci_oversold) & (d["cci"] >= cfg.cci_oversold))

    # Optional extra filters (from earlier build)
    ema_ok = (~cfg.use_ema_cross_filter) | (d["ema_fast"] > d["ema_slow"])
    sma_ok = (~cfg.use_sma_trend_filter) | (d["Close"] > d["sma_trend"])

    entry_signal = trend_ok & wpr_armed & cci_armed & wpr_conf & cci_conf & ema_ok & sma_ok

    # ---------- INDICATOR/CONDITION EXITS ----------
    # Momentum fade:
    wpr_fade = cfg.exit_on_wpr_overbought_then_below_m50 & ((d["wpr_prev"] > -20.0) & (d["wpr"] <= cfg.wpr_confirm))
    cci_fade = cfg.exit_on_cci_fall_below_p100       & ((d["cci_prev"] > cfg.cci_momentum_hi) & (d["cci"] <= cfg.cci_momentum_hi))
    # Trend break:
    ema50_break = cfg.exit_on_trend_break_ema50      & (d["Close"] < d["ema_trend50"])
    ema_cross_dn = cfg.exit_on_ema_cross_down        & (d["ema_fast"] < d["ema_slow"])
    sma_breach   = cfg.exit_on_sma_trend_breach      & (d["Close"] < d["sma_trend"])
    swing_break  = cfg.exit_on_swing_low_break       & (d["Low"] <= d["swing_low"])

    indicator_exit = wpr_fade | cci_fade | ema50_break | ema_cross_dn | sma_breach | swing_break

    in_pos = False
    entry_px = stop_px = tgt_px = 0.0
    trades = []

    idx = list(d.index)
    for i in range(len(idx)-1):
        dt, nxt = idx[i], idx[i+1]
        row, nxt_row = d.loc[dt], d.loc[nxt]

        if not in_pos:
            if entry_signal.loc[dt] and basic_liquidity_ok(row, cfg):
                # If entering on next open, avoid wild gaps
                gap = (nxt_row["Open"] - row["Close"]) / row["Close"]
                if abs(gap) <= 0.02:
                    px = float(nxt_row["Open"] if cfg.entry_on_next_open else row["Close"])
                    bits = []
                    if cfg.use_trend_filter_ema50: bits.append("Close>EMA50")
                    if cfg.use_wpr_oversold and cfg.require_wpr_confirm_above_m50: bits.append("W%R:-80→>-50")
                    elif cfg.require_wpr_confirm_above_m50: bits.append("W%R>-50")
                    if cfg.use_cci_oversold and cfg.require_cci_confirm_above_m100: bits.append("CCI:-100↑")
                    elif cfg.require_cci_confirm_above_m100: bits.append("CCI>-100")
                    if cfg.use_ema_cross_filter: bits.append(f"EMA{cfg.ema_fast}>{cfg.ema_slow}")
                    if cfg.use_sma_trend_filter: bits.append(f"Close>SMA{cfg.sma_trend_len}")
                    sig_reason = " + ".join(bits) if bits else "pullback-confirmed"
                    trades.append({
                        "ticker": ticker, "side": "BUY", "date": nxt if cfg.entry_on_next_open else dt,
                        "price": px, "shares": 0, "reason": "candidate",
                        "signal_reason": sig_reason,
                        # Rank proxy: combined oscillator strength rebound (less negative is better)
                        "score": float(row["wpr"] + row["cci"]/2.0),
                        "cci": float(row["cci"]), "wpr": float(row["wpr"]),
                        "ema_trend50": float(row["ema_trend50"]),
                        "ema_fast": float(row["ema_fast"]), "ema_slow": float(row["ema_slow"]),
                        "sma_trend": float(row["sma_trend"]),
                        "swing_low": float(row["swing_low"]),
                        "close": float(row["Close"]), "high_52w": float(row["high_52w"])
                    })
                    in_pos = True
                    entry_px = px
                    stop_px = entry_px * (1 - cfg.stop_loss_pct)
                    tgt_px  = entry_px * (1 + cfg.target_pct)

        else:
            hit = None
            exec_date = nxt if cfg.exit_on_next_open else dt

            # Hard exits on next bar range
            if nxt_row["Low"] <= stop_px and nxt_row["High"] >= tgt_px:
                hit, exec_price = "target", float(tgt_px)
            elif nxt_row["Low"] <= stop_px:
                hit, exec_price = "stop", float(stop_px)
            elif nxt_row["High"] >= tgt_px:
                hit, exec_price = "target", float(tgt_px)
            else:
                # Indicator/trend exits evaluated at dt, execute at next open/close
                if indicator_exit.loc[dt]:
                    exec_price = float(nxt_row["Open"] if cfg.exit_on_next_open else row["Close"])
                    reasons = []
                    if wpr_fade.loc[dt]: reasons.append("W%R fade (> -20 → ≤ -50)")
                    if cci_fade.loc[dt]: reasons.append("CCI fade (> +100 → ≤ +100)")
                    if ema50_break.loc[dt]: reasons.append("Close<EMA50")
                    if ema_cross_dn.loc[dt]: reasons.append(f"EMA{cfg.ema_fast}<{cfg.ema_slow}")
                    if sma_breach.loc[dt]: reasons.append(f"Close<SMA{cfg.sma_trend_len}")
                    if swing_break.loc[dt]: reasons.append(f"SwingLow({cfg.swing_lookback_n}) breach")
                    hit = "indicator_exit: " + ", ".join(reasons) if reasons else "indicator_exit"

            if hit is not None:
                trades.append({
                    "ticker": ticker, "side": "SELL", "date": exec_date,
                    "price": float(exec_price), "shares": 0, "reason": hit,
                    "signal_reason": "", "score": np.nan,
                    "cci": float(row["cci"]), "wpr": float(row["wpr"]),
                    "ema_trend50": float(row["ema_trend50"]),
                    "ema_fast": float(row["ema_fast"]), "ema_slow": float(row["ema_slow"]),
                    "sma_trend": float(row["sma_trend"]),
                    "swing_low": float(row["swing_low"]),
                    "close": float(row["Close"]), "high_52w": float(row["high_52w"])
                })
                in_pos = False
                entry_px = stop_px = tgt_px = 0.0

    # Force close if still in-position on last bar
    if in_pos:
        last_dt = d.index[-1]; row = d.loc[last_dt]
        trades.append({
            "ticker": ticker, "side": "SELL", "date": last_dt,
            "price": float(row["Close"]), "shares": 0, "reason": "final_close",
            "signal_reason": "", "score": np.nan,
            "cci": float(row["cci"]), "wpr": float(row["wpr"]),
            "ema_trend50": float(row["ema_trend50"]),
            "ema_fast": float(row["ema_fast"]), "ema_slow": float(row["ema_slow"]),
            "sma_trend": float(row["sma_trend"]),
            "swing_low": float(row["swing_low"]),
            "close": float(row["Close"]), "high_52w": float(row["high_52w"])
        })

    return pd.DataFrame(trades, columns=cols), pd.Series(dtype=float)

# =========================
# Portfolio plumbing (unchanged)
# =========================
def pick_benchmark(benchmarks: Tuple[str,...], start: str, end: Optional[str], cache_dir: str) -> Tuple[str, pd.DataFrame]:
    for t in benchmarks:
        data = fetch_prices([t], start, end, cache_dir)
        df = data.get(t)
        if df is not None and not df.empty:
            log.info("Using benchmark: %s", t); return t, df
    idx = pd.date_range(start=start, end=end or today_str(), freq="B")
    df = pd.DataFrame({"Close": np.ones(len(idx))}, index=idx)
    log.warning("No benchmark found; using synthetic flat series.")
    return "SYNTH_BENCH", df

def compute_volar_scores(end_dt: pd.Timestamp, tickers: List[str], data_map: Dict[str,pd.DataFrame], bench_df: pd.DataFrame, lookback: int) -> Dict[str, float]:
    scores = {}
    bser = bench_df["Close"].loc[:end_dt].pct_change().dropna().iloc[-lookback:]
    for t in tickers:
        df = data_map.get(t)
        if df is None or df.empty: scores[t] = 0.0; continue
        if end_dt not in df.index:
            df = df[df.index <= end_dt]
            if df.empty: scores[t] = 0.0; continue
        r = df["Close"].loc[:end_dt].pct_change().dropna().iloc[-lookback:]
        common = pd.concat([r, bser], axis=1, keys=["s","b"]).dropna()
        if common.shape[0] < max(20, int(0.4*lookback)): scores[t] = 0.0; continue
        excess = common["s"] - common["b"]
        vol = common["s"].std(ddof=0)
        scores[t] = 0.0 if vol <= 1e-8 else float((excess.mean() / vol) * math.sqrt(252.0))
    return scores

def markowitz_long_only(mu: np.ndarray, Sigma: np.ndarray) -> np.ndarray:
    n = len(mu); eps = 1e-6; Sigma = Sigma + eps*np.eye(n)
    def solve_lambda(lmbd: float, active_mask=None):
        if active_mask is None:
            A = np.block([[2*lmbd*Sigma, np.ones((n,1))],[np.ones((1,n)), np.zeros((1,1))]])
            b = np.concatenate([mu, np.array([1.0])])
            try: sol = np.linalg.solve(A, b); w = sol[:n]
            except np.linalg.LinAlgError: w = np.full(n, 1.0/n)
            return w
        idx = np.where(active_mask)[0]
        if len(idx)==0: return np.full(n, 1.0/n)
        S = Sigma[np.ix_(idx, idx)]; o = np.ones(len(idx)); m = mu[idx]
        A = np.block([[2*lmbd*S, o[:,None]],[o[None,:], np.zeros((1,1))]])
        b = np.concatenate([m, np.array([1.0])])
        try: sol = np.linalg.solve(A, b); w_sub = sol[:len(idx)]
        except np.linalg.LinAlgError: w_sub = np.full(len(idx), 1.0/len(idx))
        w = np.zeros(n); w[idx] = w_sub; return w
    best_w = np.full(n, 1.0/n); best_sr = -1e9
    for lmbd in np.logspace(-3, 3, 31):
        active = np.ones(n, dtype=bool); w = None
        for _ in range(n):
            w = solve_lambda(lmbd, active_mask=active)
            neg = w < 0
            if not neg.any(): break
            active[np.argmin(w)] = False
        if w is None: continue
        w = np.clip(w, 0, None); 
        if w.sum() <= 0: continue
        w = w / w.sum()
        mu_p = float(mu @ w); vol_p = float(np.sqrt(w @ Sigma @ w))
        if vol_p <= 1e-8: continue
        sr = mu_p / vol_p
        if sr > best_sr: best_sr = sr; best_w = w.copy()
    return best_w

def aggregate_and_apply(all_trades: pd.DataFrame, data_map: Dict[str, pd.DataFrame], bench_df: pd.DataFrame, cfg: Config):
    if all_trades.empty: return all_trades, pd.Series(dtype=float), {}
    side_order = {"BUY": 0, "SELL": 1}
    all_trades = (all_trades
        .assign(_sorder=all_trades["side"].map(side_order))
        .sort_values(by=["date", "_sorder"], kind="stable")
        .drop(columns=["_sorder"]).reset_index(drop=True))
    all_trades["date"] = pd.to_datetime(all_trades["date"])

    equity_curve = []; dates = sorted(all_trades["date"].unique().tolist())
    cash = cfg.initial_capital; open_positions = {}; completed_legs = []
    global APPLY_FEES; APPLY_FEES = cfg.apply_fees

    def _get_close_on(tkr, dt):
        df = data_map.get(tkr)
        if df is None or df.empty: return np.nan
        if dt in df.index: return float(df.loc[dt, "Close"])
        prev = df[df.index <= dt]; 
        return float(prev["Close"].iloc[-1]) if not prev.empty else np.nan

    if dates:
        seed_date = pd.to_datetime(dates[0]) - pd.Timedelta(days=1)
        equity_curve.append((seed_date, float(cash)))

    for dt in dates:
        day_trades = all_trades[all_trades["date"] == dt].copy()

        # SELL first
        for _, tr in day_trades[day_trades["side"] == "SELL"].iterrows():
            tkr = tr["ticker"]; price = float(tr["price"]); pos = open_positions.get(tkr)
            if pos is None: continue
            shares = int(pos["shares"]); turnover_sell = shares * price
            fee = calc_fees(0.0, turnover_sell)
            pnl = (price - pos["entry_px"]) * shares
            cash += (turnover_sell - fee)
            realized = pnl - fee - pos.get("buy_fee", 0.0)
            completed_legs.append({
                "ticker": tkr, "side": "SELL", "date": dt, "price": price,
                "shares": shares, "reason": tr.get("reason",""),
                "turnover": turnover_sell, "fees_inr": fee, "pnl_inr": realized,
                "cci": tr.get("cci", np.nan), "wpr": tr.get("wpr", np.nan),
                "ema_trend50": tr.get("ema_trend50", np.nan),
                "ema_fast": tr.get("ema_fast", np.nan), "ema_slow": tr.get("ema_slow", np.nan),
                "sma_trend": tr.get("sma_trend", np.nan), "swing_low": tr.get("swing_low", np.nan),
                "close": tr.get("close", np.nan), "high_52w": tr.get("high_52w", np.nan),
                "volar": tr.get("volar", np.nan), "mvo_weight": np.nan, "alloc_inr": np.nan
            })
            del open_positions[tkr]

        # BUY selection & sizing
        buys_today = day_trades[day_trades["side"] == "BUY"].copy()
        if not buys_today.empty:
            keep = []
            for _, rr in buys_today.iterrows():
                df = data_map.get(rr["ticker"])
                if df is None or df.empty or dt not in df.index: continue
                close = float(df.loc[dt, "Close"])
                hist = df["Close"].loc[:dt]
                window = hist.iloc[-cfg.filter_52w_window:] if len(hist)>=cfg.filter_52w_window else hist
                high_52w = float(window.max())
                if high_52w>0 and close >= cfg.within_pct_of_52w_high * high_52w:
                    keep.append(rr)
            buys_today = pd.DataFrame(keep) if keep else pd.DataFrame(columns=buys_today.columns)
        if not buys_today.empty:
            buys_today = buys_today[~buys_today["ticker"].isin(open_positions.keys())]
        if not buys_today.empty:
            tickers = buys_today["ticker"].tolist()
            volar_scores = compute_volar_scores(dt, tickers, data_map, bench_df, cfg.volar_lookback)
            buys_today["volar"] = buys_today["ticker"].map(volar_scores)
            buys_today = buys_today.sort_values("volar", ascending=False).reset_index(drop=True)

        slots = cfg.max_concurrent_positions - len(open_positions)
        selected = pd.DataFrame(columns=buys_today.columns)
        if slots > 0 and not buys_today.empty:
            selected = buys_today.head(min(cfg.top_k_daily, slots)).copy()

        if not selected.empty:
            names = selected["ticker"].tolist()
            rets = [data_map[t]["Close"].loc[:dt].pct_change().dropna().iloc[-cfg.volar_lookback:] for t in names]
            R = pd.concat(rets, axis=1); R.columns = names; R = R.dropna()
            if R.empty or R.shape[0] < max(20, int(0.4*cfg.volar_lookback)) or R.shape[1]==0:
                weights = np.full(len(names), 1.0/len(names))
            else:
                mu = R.mean().values; Sigma = R.cov().values
                weights = markowitz_long_only(mu, Sigma)

            deploy_cash = max(0.0, float(cash)) * float(cfg.deploy_cash_frac)
            if deploy_cash <= 0:
                log.info("No deployable cash on %s", dt.date())
            else:
                alloc = (weights / weights.sum()) * deploy_cash if weights.sum()>0 else np.full(len(names), deploy_cash/len(names))
                rank_map = {row["ticker"]: (idx+1) for idx, (_, row) in enumerate(selected.iterrows())}
                for w_amt, t in zip(alloc, names):
                    df_t = data_map[t]
                    price = float(df_t.loc[dt, "Close"] if dt in df_t.index else df_t["Close"].loc[:dt].iloc[-1])
                    shares = int(math.floor(w_amt / price))
                    if shares <= 0: continue
                    turn = shares * price
                    fee = calc_fees(turn, 0.0)
                    total_cost = turn + fee
                    if total_cost > cash:
                        shares = int(math.floor((cash - fee) / price))
                        if shares <= 0: continue
                        turn = shares * price; total_cost = turn + fee
                    cash -= total_cost
                    open_positions[t] = {"entry_date": dt, "entry_px": price, "shares": shares, "buy_fee": fee, "entry_reason": "entry"}
                    row_sel = selected[selected["ticker"]==t].iloc[0]
                    volar_val = float(row_sel.get("volar", np.nan)); rank_pos = rank_map.get(t, np.nan)
                    high_52w = float(row_sel.get("high_52w", np.nan)); close_val = float(row_sel.get("close", np.nan))
                    pct_52w = (close_val / high_52w) if (high_52w and high_52w>0) else np.nan
                    mvo_weight_today = (w_amt / deploy_cash) if deploy_cash > 0 else 0.0
                    sig_reason = row_sel.get("signal_reason", "pullback-confirmed")
                    reason_text = (f"{sig_reason}; 52w%={pct_52w:.1%} (>= {CFG.within_pct_of_52w_high:.0%}); "
                                   f"VOLAR rank {int(rank_pos)}/{len(names)} (VOLAR={volar_val:.2f}); "
                                   f"MVO weight={mvo_weight_today:.1%} of capped cash ({100*CFG.deploy_cash_frac:.0f}% of available)")
                    completed_legs.append({
                        "ticker": t, "side": "BUY", "date": dt, "price": price,
                        "shares": shares, "reason": reason_text,
                        "turnover": turn, "fees_inr": fee, "pnl_inr": 0.0,
                        "cci": float(row_sel.get("cci", np.nan)), "wpr": float(row_sel.get("wpr", np.nan)),
                        "ema_trend50": float(row_sel.get("ema_trend50", np.nan)),
                        "ema_fast": float(row_sel.get("ema_fast", np.nan)), "ema_slow": float(row_sel.get("ema_slow", np.nan)),
                        "sma_trend": float(row_sel.get("sma_trend", np.nan)), "swing_low": float(row_sel.get("swing_low", np.nan)),
                        "close": close_val, "high_52w": high_52w,
                        "volar": volar_val, "mvo_weight": float(mvo_weight_today), "alloc_inr": float(w_amt)
                    })

        # MTM valuation
        mtm = 0.0
        for _tkr, pos in open_positions.items():
            px = _get_close_on(_tkr, dt)
            if not np.isnan(px): mtm += pos["shares"] * px
        equity_curve.append((dt, float(cash + mtm)))

    eq_ser = pd.Series([e for _, e in equity_curve], index=[d for d, _ in equity_curve])
    legs_df = pd.DataFrame(completed_legs).sort_values(["date","ticker","side"]).reset_index(drop=True)

    # Roundtrips
    roundtrips = []; by_tkr_open = {}
    for _, leg in legs_df.iterrows():
        tkr = leg["ticker"]
        if leg["side"] == "BUY":
            by_tkr_open[tkr] = leg
        else:
            buy = by_tkr_open.pop(tkr, None)
            if buy is None: continue
            fees_total = float(buy.get("fees_inr", 0.0) + leg.get("fees_inr", 0.0))
            gross_pnl = (leg["price"] - buy["price"]) * buy["shares"]
            net_pnl   = gross_pnl - fees_total
            ret_pct   = (leg["price"] / buy["price"] - 1.0) * 100.0
            days_held = (pd.to_datetime(leg["date"]) - pd.to_datetime(buy["date"])).days
            roundtrips.append({
                "ticker": tkr,
                "entry_date": pd.to_datetime(buy["date"]), "entry_price": float(buy["price"]),
                "exit_date": pd.to_datetime(leg["date"]), "exit_price": float(leg["price"]),
                "days_held": int(days_held), "shares": int(buy["shares"]),
                "entry_reason": buy.get("reason",""), "exit_reason": leg.get("reason",""),
                "gross_pnl_inr": float(gross_pnl), "fees_total_inr": float(fees_total),
                "net_pnl_inr": float(net_pnl), "return_pct": float(ret_pct),
                "cci_entry": float(buy.get("cci", np.nan)), "wpr_entry": float(buy.get("wpr", np.nan)),
                "ema50_entry": float(buy.get("ema_trend50", np.nan)),
                "ema_fast_entry": float(buy.get("ema_fast", np.nan)), "ema_slow_entry": float(buy.get("ema_slow", np.nan)),
                "sma_trend_entry": float(buy.get("sma_trend", np.nan)),
                "swing_low_entry": float(buy.get("swing_low", np.nan)),
                "close_entry": float(buy.get("close", np.nan)), "high_52w_entry": float(buy.get("high_52w", np.nan)),
                "volar_entry": float(buy.get("volar", np.nan)), "mvo_weight_entry": float(buy.get("mvo_weight", np.nan)),
                "alloc_inr_entry": float(buy.get("alloc_inr", np.nan))
            })
    trips_df = pd.DataFrame(roundtrips).sort_values(["entry_date","ticker"]).reset_index(drop=True)

    metrics = compute_metrics(eq_ser, legs_df)
    return legs_df, trips_df, eq_ser, metrics

# =========================
# Metrics & plotting
# =========================
def compute_metrics(equity: pd.Series, legs_df: pd.DataFrame):
    out = {}
    if equity is None or equity.empty: return out
    eq = equity.dropna(); daily_ret = eq.pct_change().fillna(0.0)
    days = (eq.index[-1] - eq.index[0]).days or 1; years = days / 365.25
    cagr = (eq.iloc[-1] / eq.iloc[0]) ** (1/years) - 1 if years > 0 else 0.0
    sharpe = (daily_ret.mean() / daily_ret.std(ddof=0) * np.sqrt(252)) if daily_ret.std(ddof=0) > 0 else 0.0
    cummax = eq.cummax(); dd = (eq - cummax) / cummax; max_dd = dd.min()

    wins = 0
    n_sells = legs_df[legs_df["side"] == "SELL"].shape[0] if legs_df is not None and not legs_df.empty else 0
    for _, r in legs_df[legs_df["side"] == "SELL"].iterrows():
        if float(r.get("pnl_inr", 0.0)) > 0: wins += 1
    win_rate = (wins / n_sells) * 100.0 if n_sells > 0 else 0.0

    out.update({
        "start_equity_inr": float(eq.iloc[0]),
        "final_equity_inr": float(eq.iloc[-1]),
        "cagr_pct": float(cagr * 100),
        "sharpe": float(sharpe),
        "max_drawdown_pct": float(max_dd * 100),
        "win_rate_pct": float(win_rate),
        "n_trades": int(n_sells),
    })
    return out

def plot_equity(equity: pd.Series, out_path: str):
    if equity is None or equity.empty: return
    try:
        import matplotlib.pyplot as plt
        plt.figure(figsize=(10,5))
        plt.plot(equity.index, equity.values)
        plt.title("Equity Curve")
        plt.xlabel("Date"); plt.ylabel("Equity (INR)")
        plt.tight_layout(); plt.savefig(out_path); plt.close()
    except Exception:
        pass

# =========================
# Orchestration
# =========================
def backtest(cfg: Config):
    ensure_dirs(cfg.cache_dir, cfg.out_dir)
    log.info("Universe: loading static symbols...")
    symbols = load_static_symbols(cfg.static_symbols, cfg.static_symbols_path)
    log.info("Loaded %d symbols.", len(symbols))

    log.info("Data: fetching OHLCV from yfinance (adjusted)...")
    data_map = fetch_prices(symbols, cfg.start_date, cfg.end_date, cfg.cache_dir)
    log.info("Downloaded %d symbols with data.", len(data_map))

    bench_tkr, bench_df = pick_benchmark(cfg.benchmark_try, cfg.start_date, cfg.end_date, cfg.cache_dir)
    log.info("Benchmark selected: %s", bench_tkr)

    log.info(
        "Signals: Trend=%s | W%%R oversold=%s, confirm=%s | CCI oversold=%s, confirm=%s | "
        "W%%R fade exit=%s | CCI fade exit=%s | EMA50 break exit=%s | SwingLow exit=%s",
        "EMA50" if cfg.use_trend_filter_ema50 else "OFF",
        "ON" if cfg.use_wpr_oversold else "OFF",
        "ON" if cfg.require_wpr_confirm_above_m50 else "OFF",
        "ON" if cfg.use_cci_oversold else "OFF",
        "ON" if cfg.require_cci_confirm_above_m100 else "OFF",
        "ON" if cfg.exit_on_wpr_overbought_then_below_m50 else "OFF",
        "ON" if cfg.exit_on_cci_fall_below_p100 else "OFF",
        "ON" if cfg.exit_on_trend_break_ema50 else "OFF",
        "ON" if cfg.exit_on_swing_low_break else "OFF",
    )

    all_trades = []
    for i, tkr in enumerate(symbols, 1):
        df = data_map.get(tkr)
        if df is None or df.empty: continue
        tr, _ = simulate_ticker(tkr, df, cfg)
        if not tr.empty: all_trades.append(tr)
        if i % 50 == 0: log.info("  processed %d/%d tickers...", i, len(symbols))

    if not all_trades:
        log.warning("No signals. Try relaxing: require_* toggles OFF, widen thresholds, or loosen 52w filter.")
        return None, None, None, {}
    all_trades = pd.concat(all_trades, ignore_index=True)

    log.info("Portfolio: cap=%.0f%% cash; 52w≥%.0f%% high; top-%d by VOLAᵣ; MVO; max %d pos.",
             cfg.deploy_cash_frac*100, cfg.within_pct_of_52w_high*100, cfg.top_k_daily, cfg.max_concurrent_positions)
    legs_df, trips_df, equity, metrics = aggregate_and_apply(all_trades, data_map, bench_df, cfg)

    stamp = pd.Timestamp.today(tz="Asia/Kolkata").strftime("%Y%m%d_%H%M%S")
    legs_path = os.path.join(cfg.out_dir, f"trades_legs_{stamp}.csv")
    trips_path = os.path.join(cfg.out_dir, f"trades_roundtrips_{stamp}.csv")
    equity_path = os.path.join(cfg.out_dir, f"equity_{stamp}.csv")
    metrics_path = os.path.join(cfg.out_dir, f"metrics_{stamp}.json")
    eq_plot_path = os.path.join(cfg.out_dir, f"equity_{stamp}.png")

    if legs_df is not None: legs_df.to_csv(legs_path, index=False)
    if trips_df is not None: trips_df.to_csv(trips_path, index=False)
    if equity is not None:
        pd.DataFrame({"date": equity.index, "equity": equity.values}).to_csv(equity_path, index=False)
    with open(metrics_path, "w") as f: json.dump(metrics, f, indent=2)
    if cfg.plot and equity is not None: plot_equity(equity, eq_plot_path)

    log.info("=== METRICS ===\n%s", json.dumps(metrics, indent=2))
    log.info("Files written:\n  %s\n  %s\n  %s\n  %s", legs_path, trips_path, equity_path, metrics_path)
    if cfg.plot: log.info("  %s", eq_plot_path)

def main():
    global APPLY_FEES
    APPLY_FEES = bool(CFG.apply_fees)
    # Set your universe here (kept from your earlier list) or via CFG.static_symbols_path
    CFG.static_symbols = ['360ONE.NS','3MINDIA.NS','AADHARHFC.NS','AARTIIND.NS','AAVAS.NS','ABB.NS','ABBOTINDIA.NS','ABCAPITAL.NS','ABFRL.NS','ABLBL.NS','ABREL.NS','ABSLAMC.NS','ACC.NS','ACE.NS','ACMESOLAR.NS','ADANIENSOL.NS','ADANIENT.NS','ADANIGREEN.NS','ADANIPORTS.NS','ADANIPOWER.NS','AEGISLOG.NS','AEGISVOPAK.NS','AFCONS.NS','AFFLE.NS','AGARWALEYE.NS','AIAENG.NS','AIIL.NS','AJANTPHARM.NS','AKUMS.NS','AKZOINDIA.NS','ALKEM.NS','ALKYLAMINE.NS','ALOKINDS.NS','AMBER.NS','AMBUJACEM.NS','ANANDRATHI.NS','ANANTRAJ.NS','ANGELONE.NS','APARINDS.NS','APLAPOLLO.NS','APLLTD.NS','APOLLOHOSP.NS','APOLLOTYRE.NS','APTUS.NS','ARE&M.NS','ASAHIINDIA.NS','ASHOKLEY.NS','ASIANPAINT.NS','ASTERDM.NS','ASTRAL.NS','ASTRAZEN.NS','ATGL.NS','ATHERENERG.NS','ATUL.NS','AUBANK.NS','AUROPHARMA.NS','AWL.NS','AXISBANK.NS','BAJAJ-AUTO.NS','BAJAJFINSV.NS','BAJAJHFL.NS','BAJAJHLDNG.NS','BAJFINANCE.NS','BALKRISIND.NS','BALRAMCHIN.NS','BANDHANBNK.NS','BANKBARODA.NS','BANKINDIA.NS','BASF.NS','BATAINDIA.NS','BAYERCROP.NS','BBTC.NS','BDL.NS','BEL.NS','BEML.NS','BERGEPAINT.NS','BHARATFORG.NS','BHARTIARTL.NS','BHARTIHEXA.NS','BHEL.NS','BIKAJI.NS','BIOCON.NS','BLS.NS','BLUEDART.NS','BLUEJET.NS','BLUESTARCO.NS','BOSCHLTD.NS','BPCL.NS','BRIGADE.NS','BRITANNIA.NS','BSE.NS','BSOFT.NS','CAMPUS.NS','CAMS.NS','CANBK.NS','CANFINHOME.NS','CAPLIPOINT.NS','CARBORUNIV.NS','CASTROLIND.NS','CCL.NS','CDSL.NS','CEATLTD.NS','CENTRALBK.NS','CENTURYPLY.NS','CERA.NS','CESC.NS','CGCL.NS','CGPOWER.NS','CHALET.NS','CHAMBLFERT.NS','CHENNPETRO.NS','CHOICEIN.NS','CHOLAFIN.NS','CHOLAHLDNG.NS','CIPLA.NS','CLEAN.NS','COALINDIA.NS','COCHINSHIP.NS','COFORGE.NS','COHANCE.NS','COLPAL.NS','CONCOR.NS','CONCORDBIO.NS','COROMANDEL.NS','CRAFTSMAN.NS','CREDITACC.NS','CRISIL.NS','CROMPTON.NS','CUB.NS','CUMMINSIND.NS','CYIENT.NS','DABUR.NS','DALBHARAT.NS','DATAPATTNS.NS','DBREALTY.NS','DCMSHRIRAM.NS','DEEPAKFERT.NS','DEEPAKNTR.NS','DELHIVERY.NS','DEVYANI.NS','DIVISLAB.NS','DIXON.NS','DLF.NS','DMART.NS','DOMS.NS','DRREDDY.NS','ECLERX.NS','EICHERMOT.NS','EIDPARRY.NS','EIHOTEL.NS','ELECON.NS','ELGIEQUIP.NS','EMAMILTD.NS','EMCURE.NS','ENDURANCE.NS','ENGINERSIN.NS','ENRIN.NS','ERIS.NS','ESCORTS.NS','ETERNAL.NS','EXIDEIND.NS','FACT.NS','FEDERALBNK.NS','FINCABLES.NS','FINPIPE.NS','FIRSTCRY.NS','FIVESTAR.NS','FLUOROCHEM.NS','FORCEMOT.NS','FORTIS.NS','FSL.NS','GAIL.NS','GESHIP.NS','GICRE.NS','GILLETTE.NS','GLAND.NS','GLAXO.NS','GLENMARK.NS','GMDCLTD.NS','GMRAIRPORT.NS','GODFRYPHLP.NS','GODIGIT.NS','GODREJAGRO.NS','GODREJCP.NS','GODREJIND.NS','GODREJPROP.NS','GPIL.NS','GRANULES.NS','GRAPHITE.NS','GRASIM.NS','GRAVITA.NS','GRSE.NS','GSPL.NS','GUJGASLTD.NS','GVT&D.NS','HAL.NS','HAPPSTMNDS.NS','HAVELLS.NS','HBLENGINE.NS','HCLTECH.NS','HDFCAMC.NS','HDFCBANK.NS','HDFCLIFE.NS','HEG.NS','HEROMOTOCO.NS','HEXT.NS','HFCL.NS','HINDALCO.NS','HINDCOPPER.NS','HINDPETRO.NS','HINDUNILVR.NS','HINDZINC.NS','HOMEFIRST.NS','HONASA.NS','HONAUT.NS','HSCL.NS','HUDCO.NS','HYUNDAI.NS','ICICIBANK.NS','ICICIGI.NS','ICICIPRULI.NS','IDBI.NS','IDEA.NS','IDFCFIRSTB.NS','IEX.NS','IFCI.NS','IGIL.NS','IGL.NS','IIFL.NS','IKS.NS','INDGN.NS','INDHOTEL.NS','INDIACEM.NS','INDIAMART.NS','INDIANB.NS','INDIGO.NS','INDUSINDBK.NS','INDUSTOWER.NS','INFY.NS','INOXINDIA.NS','INOXWIND.NS','INTELLECT.NS','IOB.NS','IOC.NS','IPCALAB.NS','IRB.NS','IRCON.NS','IRCTC.NS','IREDA.NS','IRFC.NS','ITC.NS','ITCHOTELS.NS','ITI.NS','J&KBANK.NS','JBCHEPHARM.NS','JBMA.NS','JINDALSAW.NS','JINDALSTEL.NS','JIOFIN.NS','JKCEMENT.NS','JKTYRE.NS','JMFINANCIL.NS','JPPOWER.NS','JSL.NS','JSWENERGY.NS','JSWINFRA.NS','JSWSTEEL.NS','JUBLFOOD.NS','JUBLINGREA.NS','JUBLPHARMA.NS','JWL.NS','JYOTHYLAB.NS','JYOTICNC.NS','KAJARIACER.NS','KALYANKJIL.NS','KARURVYSYA.NS','KAYNES.NS','KEC.NS','KEI.NS','KFINTECH.NS','KIMS.NS','KIRLOSBROS.NS','KIRLOSENG.NS','KOTAKBANK.NS','KPIL.NS','KPITTECH.NS','KPRMILL.NS','KSB.NS','LALPATHLAB.NS','LATENTVIEW.NS','LAURUSLABS.NS','LEMONTREE.NS','LICHSGFIN.NS','LICI.NS','LINDEINDIA.NS','LLOYDSME.NS','LODHA.NS','LT.NS','LTF.NS','LTFOODS.NS','LTIM.NS','LTTS.NS','LUPIN.NS','M&M.NS','M&MFIN.NS','MAHABANK.NS','MAHSCOOTER.NS','MAHSEAMLES.NS','MANAPPURAM.NS','MANKIND.NS','MANYAVAR.NS','MAPMYINDIA.NS','MARICO.NS','MARUTI.NS','MAXHEALTH.NS','MAZDOCK.NS','MCX.NS','MEDANTA.NS','METROPOLIS.NS','MFSL.NS','MGL.NS','MINDACORP.NS','MMTC.NS','MOTHERSON.NS','MOTILALOFS.NS','MPHASIS.NS','MRF.NS','MRPL.NS','MSUMI.NS','MUTHOOTFIN.NS','NAM-INDIA.NS','NATCOPHARM.NS','NATIONALUM.NS','NAUKRI.NS','NAVA.NS','NAVINFLUOR.NS','NBCC.NS','NCC.NS','NESTLEIND.NS','NETWEB.NS','NEULANDLAB.NS','NEWGEN.NS','NH.NS','NHPC.NS','NIACL.NS','NIVABUPA.NS','NLCINDIA.NS','NMDC.NS','NSLNISP.NS','NTPC.NS','NTPCGREEN.NS','NUVAMA.NS','NUVOCO.NS','NYKAA.NS','OBEROIRLTY.NS','OFSS.NS','OIL.NS','OLAELEC.NS','OLECTRA.NS','ONESOURCE.NS','ONGC.NS','PAGEIND.NS','PATANJALI.NS','PAYTM.NS','PCBL.NS','PERSISTENT.NS','PETRONET.NS','PFC.NS','PFIZER.NS','PGEL.NS','PGHH.NS','PHOENIXLTD.NS','PIDILITIND.NS','PIIND.NS','PNB.NS','PNBHOUSING.NS','POLICYBZR.NS','POLYCAB.NS','POLYMED.NS','POONAWALLA.NS','POWERGRID.NS','POWERINDIA.NS','PPLPHARMA.NS','PRAJIND.NS','PREMIERENE.NS','PRESTIGE.NS','PTCIL.NS','PVRINOX.NS','RADICO.NS','RAILTEL.NS','RAINBOW.NS','RAMCOCEM.NS','RBLBANK.NS','RCF.NS','RECLTD.NS','REDINGTON.NS','RELIANCE.NS','RELINFRA.NS','RHIM.NS','RITES.NS','RKFORGE.NS','RPOWER.NS','RRKABEL.NS','RVNL.NS','SAGILITY.NS','SAIL.NS','SAILIFE.NS','SAMMAANCAP.NS','SAPPHIRE.NS','SARDAEN.NS','SAREGAMA.NS','SBFC.NS','SBICARD.NS','SBILIFE.NS','SBIN.NS','SCHAEFFLER.NS','SCHNEIDER.NS','SCI.NS','SHREECEM.NS','SHRIRAMFIN.NS','SHYAMMETL.NS','SIEMENS.NS','SIGNATURE.NS','SJVN.NS','SKFINDIA.NS','SOBHA.NS','SOLARINDS.NS','SONACOMS.NS','SONATSOFTW.NS','SRF.NS','STARHEALTH.NS','SUMICHEM.NS','SUNDARMFIN.NS','SUNDRMFAST.NS','SUNPHARMA.NS','SUNTV.NS','SUPREMEIND.NS','SUZLON.NS','SWANCORP.NS','SWIGGY.NS','SYNGENE.NS','SYRMA.NS','TARIL.NS','TATACHEM.NS','TATACOMM.NS','TATACONSUM.NS','TATAELXSI.NS','TATAINVEST.NS','TATAMOTORS.NS','TATAPOWER.NS','TATASTEEL.NS','TATATECH.NS','TBOTEK.NS','TCS.NS','TECHM.NS','TECHNOE.NS','TEJASNET.NS','THELEELA.NS','THERMAX.NS','TIINDIA.NS','TIMKEN.NS','TITAGARH.NS','TITAN.NS','TORNTPHARM.NS','TORNTPOWER.NS','TRENT.NS','TRIDENT.NS','TRITURBINE.NS','TRIVENI.NS','TTML.NS','TVSMOTOR.NS','UBL.NS','UCOBANK.NS','ULTRACEMCO.NS','UNIONBANK.NS','UNITDSPR.NS','UNOMINDA.NS','UPL.NS','USHAMART.NS','UTIAMC.NS','VBL.NS','VEDL.NS','VENTIVE.NS','VGUARD.NS','VIJAYA.NS','VMM.NS','VOLTAS.NS','VTL.NS','WAAREEENER.NS','WELCORP.NS','WELSPUNLIV.NS','WHIRLPOOL.NS','WIPRO.NS','WOCKPHARMA.NS','YESBANK.NS','ZEEL.NS','ZENSARTECH.NS','ZENTEC.NS','ZFCVINDIA.NS','ZYDUSLIFE.NS']
    # CFG.static_symbols_path = "nifty500.txt"
    backtest(CFG)

if __name__ == "__main__":
    main()


2025-10-13 19:04:36 | INFO | Universe: loading static symbols...
2025-10-13 19:04:36 | INFO | Loaded 500 symbols.
2025-10-13 19:04:36 | INFO | Data: fetching OHLCV from yfinance (adjusted)...
2025-10-13 19:04:39 | ERROR | 
1 Failed download:
2025-10-13 19:04:39 | ERROR | ['ABLBL.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2025-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1420050600, endDate = 1735669800")')
2025-10-13 19:04:45 | ERROR | 
1 Failed download:
2025-10-13 19:04:45 | ERROR | ['AEGISVOPAK.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2025-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1420050600, endDate = 1735669800")')
2025-10-13 19:04:46 | ERROR | 
1 Failed download:
2025-10-13 19:04:46 | ERROR | ['AGARWALEYE.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2025-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1420050600